In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install ultralytics
!pip install pyngrok

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [3]:
import torch
from ultralytics import YOLO
from PIL import Image
import requests
from io import BytesIO
from flask import Flask, request, jsonify, send_file
from pyngrok import ngrok, conf
import os
import numpy as np
import cv2
import base64

app = Flask(__name__)

# YOLO model path
MODEL_PATH = "/content/drive/MyDrive/DoAnCV/best.pt"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load YOLO model
model = YOLO(MODEL_PATH)

# Vietnamese class names mapping
class_vi = [
    'áo tay ngắn',
    'áo tay dài',
    'áo khoác tay ngắn',
    'áo khoác tay dài',
    'áo ghi-lê, áo ba lỗ, áo vest',
    'áo hai dây',
    'quần short',
    'quần dài',
    'váy ngắn',
    'đầm tay ngắn',
    'đầm tay dài',
    'đầm sát nách',
    'đầm hai dây'
]

# HTML content embedded as a string
HTML_CONTENT = """
<!DOCTYPE html>
<html>
<head>
    <title>YOLO Object Detection Demo</title>
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <script src="https://unpkg.com/react@18.2.0/umd/react.production.min.js"></script>
    <script src="https://unpkg.com/react-dom@18.2.0/umd/react-dom.production.min.js"></script>
    <script src="https://unpkg.com/@babel/standalone@7.25.7/babel.min.js"></script>
    <script src="https://cdn.tailwindcss.com"></script>
    <style>
        .spinner {
            border: 4px solid rgba(0, 0, 0, 0.1);
            border-left-color: #3b82f6;
            border-radius: 50%;
            width: 24px;
            height: 24px;
            animation: spin 1s linear infinite;
            display: inline-block;
        }
        @keyframes spin {
            to { transform: rotate(360deg); }
        }
    </style>
</head>
<body>
    <div id="root"></div>
    <script type="text/babel">
        class ErrorBoundary extends React.Component {
            state = { error: null };
            static getDerivedStateFromError(error) {
                return { error: error.message };
            }
            render() {
                if (this.state.error) {
                    return (
                        <div className="text-red-500 text-center p-4">
                            Something went wrong: {this.state.error}
                        </div>
                    );
                }
                return this.props.children;
            }
        }

        function App() {
            const [imageUrl, setImageUrl] = React.useState('');
            const [imageFile, setImageFile] = React.useState(null);
            const [result, setResult] = React.useState(null);
            const [error, setError] = React.useState(null);
            const [imagePreview, setImagePreview] = React.useState(null);
            const [isLoading, setIsLoading] = React.useState(false);

            const handleUrlChange = (e) => {
                setImageUrl(e.target.value);
                setImageFile(null);
                setImagePreview(e.target.value);
                setResult(null);
                setError(null);
            };

            const handleFileChange = (e) => {
                const file = e.target.files[0];
                if (file) {
                    setImageFile(file);
                    setImageUrl('');
                    setImagePreview(URL.createObjectURL(file));
                    setResult(null);
                    setError(null);
                }
            };

            const handlePredict = async () => {
                if (!imageFile && !imageUrl) {
                    setError('Please provide an image file or URL');
                    return;
                }
                setIsLoading(true);
                const formData = new FormData();
                if (imageFile) {
                    formData.append('file', imageFile);
                } else if (imageUrl) {
                    formData.append('url', imageUrl);
                }

                try {
                    const response = await fetch('/predict', {
                        method: 'POST',
                        body: formData
                    });
                    const data = await response.json();
                    if (data.error) {
                        setError(data.error);
                        setResult(null);
                    } else {
                        setResult(data);
                        setError(null);
                    }
                } catch (err) {
                    setError('Failed to connect to the server');
                    setResult(null);
                } finally {
                    setIsLoading(false);
                }
            };

            return (
                <div className="min-h-screen bg-gray-100 flex items-center justify-center p-4 sm:p-6">
                    <div className="w-full max-w-lg bg-white rounded-lg shadow-lg p-6 sm:p-8">
                        <h1 className="text-2xl sm:text-3xl font-bold mb-6 text-center text-gray-800">
                            YOLO Object Detection Demo
                        </h1>
                        <div className="space-y-4">
                            <input
                                type="text"
                                placeholder="Paste image URL here"
                                value={imageUrl}
                                onChange={handleUrlChange}
                                className="w-full p-3 border rounded-lg focus:ring-2 focus:ring-blue-500 focus:border-blue-500 transition"
                            />
                            <input
                                type="file"
                                accept="image/*"
                                onChange={handleFileChange}
                                className="w-full text-sm text-gray-500 file:mr-4 file:py-2 file:px-4 file:rounded file:border-0 file:bg-blue-50 file:text-blue-700 hover:file:bg-blue-100"
                            />
                            <button
                                onClick={handlePredict}
                                disabled={isLoading}
                                className={`w-full py-3 rounded-lg font-semibold text-white transition duration-300 ${
                                    isLoading ? 'bg-blue-400 cursor-not-allowed' : 'bg-blue-500 hover:bg-blue-600'
                                } flex items-center justify-center`}
                            >
                                {isLoading ? (
                                    <>
                                        <span className="spinner mr-2"></span>
                                        Detecting...
                                    </>
                                ) : (
                                    'Detect'
                                )}
                            </button>
                            {imagePreview && !result && (
                                <div className="mt-4 flex justify-center">
                                    <img
                                        src={imagePreview}
                                        alt="Preview"
                                        className="-tw-w-full tw-h-auto tw-rounded-lg tw-shadow-md tw-max-h-64"
                                    />
                                </div>
                            )}
                            {result && result.annotated_image && (
                                <div className="mt-4 flex justify-center">
                                    <img
                                        src={result.annotated_image}
                                        alt="Annotated"
                                        className="tw-w-full tw-h-auto tw-rounded-lg tw-shadow-md tw-max-h-64"
                                    />
                                </div>
                            )}
                            {error && (
                                <div className="text-red-500 mt-4 text-center font-medium">
                                    Error: {error}
                                </div>
                            )}
                            {result && result.detections && (
                                <div className="mt-6 text-center bg-gray-50 p-4 rounded-lg">
                                    <h3 className="font-medium text-gray-700">Detections:</h3>
                                    <ul className="mt-2 text-sm text-gray-600 space-y-1">
                                        {result.detections.map((det, index) => (
                                            <li key={index}>
                                                {det.class} (Confidence: {det.confidence}, BBox: {det.bbox.join(', ')})
                                            </li>
                                        ))}
                                    </ul>
                                </div>
                            )}
                        </div>
                    </div>
                </div>
            );
        }

        const root = ReactDOM.createRoot(document.getElementById('root'));
        root.render(
            <ErrorBoundary>
                <App />
            </ErrorBoundary>
        );
    </script>
</body>
</html>
"""

# Write HTML content to file
with open("index.html", "w") as f:
    f.write(HTML_CONTENT)

def predict_image(image):
    # Convert PIL image to OpenCV format
    image_np = np.array(image)
    image_np = cv2.cvtColor(image_np, cv2.COLOR_RGB2BGR)

    # Run YOLO prediction
    results = model(image_np)

    # Process results
    detections = []
    for result in results:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy().astype(int)
            conf = float(box.conf.cpu().numpy())
            cls = int(box.cls.cpu().numpy())
            class_name = class_vi[cls] if cls < len(class_vi) else f'class_{cls}'
            detections.append({
                "class": class_name,
                "confidence": f"{conf*100:.2f}%",
                "bbox": [int(x1), int(y1), int(x2), int(y2)]
            })

        # Draw bounding boxes on image
        annotated_image = result.plot()
        # Convert back to PIL for base64 encoding
        annotated_image = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2RGB)
        _, buffer = cv2.imencode('.jpg', annotated_image)
        img_base64 = base64.b64encode(buffer).decode('utf-8')

    return detections, img_base64

@app.route('/predict', methods=['POST'])
def predict():
    try:
        if 'file' in request.files:
            file = request.files['file']
            image = Image.open(file.stream).convert('RGB')
        elif 'url' in request.form:
            url = request.form['url']
            response = requests.get(url)
            response.raise_for_status()
            image = Image.open(BytesIO(response.content)).convert('RGB')
        else:
            return jsonify({"error": "No file or URL provided"}), 400

        detections, annotated_image = predict_image(image)

        result = {
            "detections": detections,
            "annotated_image": f"data:image/jpeg;base64,{annotated_image}"
        }
        return jsonify(result)
    except Exception as e:
        return jsonify({"error": str(e)}), 500

@app.route('/')
@app.route('/index.html')
def serve_html():
    return send_file("index.html")

def main():
    authtoken = Auth here"
    conf.get_default().auth_token = authtoken

    public_url = ngrok.connect(5000, proto="http").public_url
    print(f" * ngrok tunnel available at: {public_url}")
    print(f" * Open the frontend at: {public_url}/index.html")

    app.run(host="0.0.0.0", port=5000)

if __name__ == "__main__":
    main()

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
 * ngrok tunnel available at: https://86d8-34-87-122-96.ngrok-free.app
 * Open the frontend at: https://86d8-34-87-122-96.ngrok-free.app/index.html
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.2:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 18:10:02] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 18:10:04] "GET /favicon.ico HTTP/1.1" 404 -



0: 640x640 1 vest, 1 vest dress, 64.3ms
Speed: 26.9ms preprocess, 64.3ms inference, 341.0ms postprocess per image at shape (1, 3, 640, 640)


<ipython-input-3-ed84ff49e0aa>:252: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  conf = float(box.conf.cpu().numpy())
<ipython-input-3-ed84ff49e0aa>:253: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cls = int(box.cls.cpu().numpy())
INFO:werkzeug:127.0.0.1 - - [04/Jun/2025 18:10:15] "POST /predict HTTP/1.1" 200 -
